In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import preprocessing
from sklearn.preprocessing import Normalizer,MinMaxScaler
from scipy.cluster.hierarchy import dendrogram, ward
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import silhouette_samples
from matplotlib import cm
from sklearn.manifold import TSNE

import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install openpyxl

In [3]:
data = pd.read_excel('innercity.xlsx')
data

,cid,dayhours,price,room_bed,room_bath,living_measure,lot_measure,ceil,coast,sight,...,basement,yr_built,yr_renovated,zipcode,lat,long,living_measure15,lot_measure15,furnished,total_area
0,3876100940,20150427T000000,600000,4.0,1.75,3050.0,9440.0,1,0,0.0,...,1250.0,1966,0,98034,47.7228,-122.183,2020.0,8660.0,0.0,12490
1,3145600250,20150317T000000,190000,2.0,1.00,670.0,3101.0,1,0,0.0,...,0.0,1948,0,98118,47.5546,-122.274,1660.0,4100.0,0.0,3771
2,7129303070,20140820T000000,735000,4.0,2.75,3040.0,2415.0,2,1,4.0,...,0.0,1966,0,98118,47.5188,-122.256,2620.0,2433.0,0.0,5455
3,7338220280,20141010T000000,257000,3.0,2.50,1740.0,3721.0,2,0,0.0,...,0.0,2009,0,98002,47.3363,-122.213,2030.0,3794.0,0.0,5461
4,7950300670,20150218T000000,450000,2.0,1.00,1120.0,4590.0,1,0,0.0,...,0.0,1924,0,98118,47.5663,-122.285,1120.0,5100.0,0.0,5710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,203600600,20150310T000000,685530,4.0,2.50,3130.0,60467.0,2,0,0.0,...,0.0,1996,0,98014,47.6618,-121.962,2780.0,44224.0,1.0,63597
21609,625049281,20140521T000000,535000,2.0,1.00,1030.0,4841.0,1,0,0.0,...,110.0,1939,0,98103,47.6860,-122.341,1530.0,4944.0,0.0,5871
21610,424069018,20140905T000000,998000,3.0,3.75,3710.0,34412.0,2,0,0.0,...,800.0,1978,0,98075,47.5888,-122.04,2390.0,34412.0,1.0,38122
21611,7258200055,20150206T000000,262000,4.0,2.50,1560.0,7800.0,2,0,0.0,...,0.0,1997,0,98168,47.5140,-122.316,1160.0,7800.0,0.0,9360


In [4]:
data_columns = data.columns
data_columns

Index(['cid', 'dayhours', 'price', 'room_bed', 'room_bath', 'living_measure',
       'lot_measure', 'ceil', 'coast', 'sight', 'condition', 'quality',
       'ceil_measure', 'basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'living_measure15', 'lot_measure15', 'furnished',
       'total_area'],
      dtype='object')

In [5]:
%%time
start='\033[1m'
end='\033[0;0m'

Wall time: 0 ns


In [6]:
print(start+'DATASET SUMMARY:'+end,'\n')
print(data.info(),'\n')

DATASET SUMMARY: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cid               21613 non-null  int64  
 1   dayhours          21613 non-null  object 
 2   price             21613 non-null  int64  
 3   room_bed          21505 non-null  float64
 4   room_bath         21505 non-null  float64
 5   living_measure    21596 non-null  float64
 6   lot_measure       21571 non-null  float64
 7   ceil              21571 non-null  object 
 8   coast             21612 non-null  object 
 9   sight             21556 non-null  float64
 10  condition         21556 non-null  object 
 11  quality           21612 non-null  float64
 12  ceil_measure      21612 non-null  float64
 13  basement          21612 non-null  float64
 14  yr_built          21612 non-null  object 
 15  yr_renovated      21613 non-null  int64  
 16  zipcode           216

#  Variable transformation

In [12]:
data.coast.value_counts()

0    21421
1      161
$       30
Name: coast, dtype: int64

In [16]:
# columns where the $ characater is found
data.select_dtypes([np.object]).apply(lambda x: x.str.contains('$').any())

dayhours      True
ceil          True
coast         True
condition     True
yr_built      True
long          True
total_area    True
dtype: bool

In [21]:
test = data.select_dtypes([np.object]).apply(lambda x: x.str.contains('$').any())
character_list =test.index.values.tolist()
character_list

['dayhours', 'ceil', 'coast', 'condition', 'yr_built', 'long', 'total_area']

In [26]:
#function replacing character value
def replace_char_in_column(df, column_name, char_to_replace, replacement_char):
    df[column_name] = df[column_name].apply(lambda x: x.replace(char_to_replace, replacement_char))
    return df

In [28]:
replace_char_in_column(data, character_list, "$", np.nan)

,cid,dayhours,price,room_bed,room_bath,living_measure,lot_measure,ceil,coast,sight,...,basement,yr_built,yr_renovated,zipcode,lat,long,living_measure15,lot_measure15,furnished,total_area
0,3876100940,20150427T000000,600000,4.0,1.75,3050.0,9440.0,1.0,0.0,0.0,...,1250.0,1966.0,0,98034,47.7228,-122.183,2020.0,8660.0,0.0,12490.0
1,3145600250,20150317T000000,190000,2.0,1.00,670.0,3101.0,1.0,0.0,0.0,...,0.0,1948.0,0,98118,47.5546,-122.274,1660.0,4100.0,0.0,3771.0
2,7129303070,20140820T000000,735000,4.0,2.75,3040.0,2415.0,2.0,1.0,4.0,...,0.0,1966.0,0,98118,47.5188,-122.256,2620.0,2433.0,0.0,5455.0
3,7338220280,20141010T000000,257000,3.0,2.50,1740.0,3721.0,2.0,0.0,0.0,...,0.0,2009.0,0,98002,47.3363,-122.213,2030.0,3794.0,0.0,5461.0
4,7950300670,20150218T000000,450000,2.0,1.00,1120.0,4590.0,1.0,0.0,0.0,...,0.0,1924.0,0,98118,47.5663,-122.285,1120.0,5100.0,0.0,5710.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,203600600,20150310T000000,685530,4.0,2.50,3130.0,60467.0,2.0,0.0,0.0,...,0.0,1996.0,0,98014,47.6618,-121.962,2780.0,44224.0,1.0,63597.0
21609,625049281,20140521T000000,535000,2.0,1.00,1030.0,4841.0,1.0,0.0,0.0,...,110.0,1939.0,0,98103,47.6860,-122.341,1530.0,4944.0,0.0,5871.0
21610,424069018,20140905T000000,998000,3.0,3.75,3710.0,34412.0,2.0,0.0,0.0,...,800.0,1978.0,0,98075,47.5888,-122.040,2390.0,34412.0,1.0,38122.0
21611,7258200055,20150206T000000,262000,4.0,2.50,1560.0,7800.0,2.0,0.0,0.0,...,0.0,1997.0,0,98168,47.5140,-122.316,1160.0,7800.0,0.0,9360.0


In [29]:
data.coast.value_counts()

0.0    21421
1.0      161
Name: coast, dtype: int64

In [30]:
data.select_dtypes([np.object]).apply(lambda x: x.str.contains('$').any())

dayhours    True
dtype: bool

In [36]:
data.dayhours.str.contains("$").count()

21613

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cid               21613 non-null  int64  
 1   dayhours          21613 non-null  object 
 2   price             21613 non-null  int64  
 3   room_bed          21505 non-null  float64
 4   room_bath         21505 non-null  float64
 5   living_measure    21596 non-null  float64
 6   lot_measure       21571 non-null  float64
 7   ceil              21541 non-null  float64
 8   coast             21582 non-null  float64
 9   sight             21556 non-null  float64
 10  condition         21528 non-null  float64
 11  quality           21612 non-null  float64
 12  ceil_measure      21612 non-null  float64
 13  basement          21612 non-null  float64
 14  yr_built          21598 non-null  float64
 15  yr_renovated      21613 non-null  int64  
 16  zipcode           21613 non-null  int64 

In [38]:
data['condition']=data['condition'].astype('int', errors='ignore')
data['yr_built']=data['yr_built'].astype('int',errors='ignore')

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cid               21613 non-null  int64  
 1   dayhours          21613 non-null  object 
 2   price             21613 non-null  int64  
 3   room_bed          21505 non-null  float64
 4   room_bath         21505 non-null  float64
 5   living_measure    21596 non-null  float64
 6   lot_measure       21571 non-null  float64
 7   ceil              21541 non-null  float64
 8   coast             21582 non-null  float64
 9   sight             21556 non-null  float64
 10  condition         21528 non-null  float64
 11  quality           21612 non-null  float64
 12  ceil_measure      21612 non-null  float64
 13  basement          21612 non-null  float64
 14  yr_built          21598 non-null  float64
 15  yr_renovated      21613 non-null  int64  
 16  zipcode           21613 non-null  int64 

In [40]:
print(start+'Checking for Null values in the dataframe:'+end,'\n',data.isnull().sum(),'\n')

Checking for Null values in the dataframe: 
 cid                   0
dayhours              0
price                 0
room_bed            108
room_bath           108
living_measure       17
lot_measure          42
ceil                 72
coast                31
sight                57
condition            85
quality               1
ceil_measure          1
basement              1
yr_built             15
yr_renovated          0
zipcode               0
lat                   0
long                 34
living_measure15    166
lot_measure15        29
furnished            29
total_area           68
dtype: int64 



# Dealing with missing values using simple imputer

In [54]:
#function Filling the missing values in categorical variable using Mode
from typing import List

def fill_nan_with_mode(df, column_names:List):
    for column in column_names:
        mode = df[column].mode().values[0]
        df[column].fillna(mode, inplace=True)
    return df

In [56]:
# filling nan for categorical data with mode
categorical_columns = ['furnished','quality', 'condition', 'coast', 'ceil', 'sight', 'yr_renovated']

fill_nan_with_mode(data, categorical_columns)
print(start+'Checking for Null values in the dataframe:'+end,'\n',data.isnull().sum(),'\n')

Checking for Null values in the dataframe: 
 cid                   0
dayhours              0
price                 0
room_bed            108
room_bath           108
living_measure       17
lot_measure          42
ceil                  0
coast                 0
sight                 0
condition             0
quality               0
ceil_measure          1
basement              1
yr_built             15
yr_renovated          0
zipcode               0
lat                   0
long                 34
living_measure15    166
lot_measure15        29
furnished             0
total_area           68
dtype: int64 



In [58]:
# Filling the missing value in numerical variables using Median
def fill_nan_with_median(df, column_names:List):
    for column in column_names:
        median = df[column].median()
        df[column].fillna(median, inplace=True)
    return df

In [67]:
# create a list of numerical columns
numerical_columns = data.drop(categorical_columns,axis=1).columns.tolist()  

# remove additional columns
[numerical_columns.remove(elem) for elem in ['cid', 'dayhours']]

numerical_columns

['price',
 'room_bed',
 'room_bath',
 'living_measure',
 'lot_measure',
 'ceil_measure',
 'basement',
 'yr_built',
 'zipcode',
 'lat',
 'long',
 'living_measure15',
 'lot_measure15',
 'total_area']

In [57]:
data.median()

cid                 3.904930e+09
price               4.500000e+05
room_bed            3.000000e+00
room_bath           2.250000e+00
living_measure      1.910000e+03
lot_measure         7.618000e+03
ceil                1.500000e+00
coast               0.000000e+00
sight               0.000000e+00
condition           3.000000e+00
quality             7.000000e+00
ceil_measure        1.560000e+03
basement            0.000000e+00
yr_built            1.975000e+03
yr_renovated        0.000000e+00
zipcode             9.806500e+04
lat                 4.757180e+01
long               -1.222300e+02
living_measure15    1.840000e+03
lot_measure15       7.620000e+03
furnished           0.000000e+00
total_area          9.575000e+03
dtype: float64

In [69]:
fill_nan_with_median(data, numerical_columns)

,cid,dayhours,price,room_bed,room_bath,living_measure,lot_measure,ceil,coast,sight,...,basement,yr_built,yr_renovated,zipcode,lat,long,living_measure15,lot_measure15,furnished,total_area
0,3876100940,20150427T000000,600000,4.0,1.75,3050.0,9440.0,1.0,0.0,0.0,...,1250.0,1966.0,0,98034,47.7228,-122.183,2020.0,8660.0,0.0,12490.0
1,3145600250,20150317T000000,190000,2.0,1.00,670.0,3101.0,1.0,0.0,0.0,...,0.0,1948.0,0,98118,47.5546,-122.274,1660.0,4100.0,0.0,3771.0
2,7129303070,20140820T000000,735000,4.0,2.75,3040.0,2415.0,2.0,1.0,4.0,...,0.0,1966.0,0,98118,47.5188,-122.256,2620.0,2433.0,0.0,5455.0
3,7338220280,20141010T000000,257000,3.0,2.50,1740.0,3721.0,2.0,0.0,0.0,...,0.0,2009.0,0,98002,47.3363,-122.213,2030.0,3794.0,0.0,5461.0
4,7950300670,20150218T000000,450000,2.0,1.00,1120.0,4590.0,1.0,0.0,0.0,...,0.0,1924.0,0,98118,47.5663,-122.285,1120.0,5100.0,0.0,5710.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,203600600,20150310T000000,685530,4.0,2.50,3130.0,60467.0,2.0,0.0,0.0,...,0.0,1996.0,0,98014,47.6618,-121.962,2780.0,44224.0,1.0,63597.0
21609,625049281,20140521T000000,535000,2.0,1.00,1030.0,4841.0,1.0,0.0,0.0,...,110.0,1939.0,0,98103,47.6860,-122.341,1530.0,4944.0,0.0,5871.0
21610,424069018,20140905T000000,998000,3.0,3.75,3710.0,34412.0,2.0,0.0,0.0,...,800.0,1978.0,0,98075,47.5888,-122.040,2390.0,34412.0,1.0,38122.0
21611,7258200055,20150206T000000,262000,4.0,2.50,1560.0,7800.0,2.0,0.0,0.0,...,0.0,1997.0,0,98168,47.5140,-122.316,1160.0,7800.0,0.0,9360.0


In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cid               21613 non-null  int64  
 1   dayhours          21613 non-null  object 
 2   price             21613 non-null  int64  
 3   room_bed          21613 non-null  float64
 4   room_bath         21613 non-null  float64
 5   living_measure    21613 non-null  float64
 6   lot_measure       21613 non-null  float64
 7   ceil              21613 non-null  float64
 8   coast             21613 non-null  float64
 9   sight             21613 non-null  float64
 10  condition         21613 non-null  float64
 11  quality           21613 non-null  float64
 12  ceil_measure      21613 non-null  float64
 13  basement          21613 non-null  float64
 14  yr_built          21613 non-null  float64
 15  yr_renovated      21613 non-null  int64  
 16  zipcode           21613 non-null  int64 

In [71]:
df = data.copy()